Ovaj notebook priprema podatke za klasifikaciju. Pravi različite skupove u zavisnosti od različitih metoda predobrade podataka. Skupovi se kreću od najjednostavnijih gde gotovo nikakva predobrada nije izvršena, do komplikovanijih gde je više metoda predobrade primenjeno.

In [1]:
import pandas as pd

In [2]:
# Ucitavanje podataka

df_merged = pd.read_csv('./data/annotation_merged.csv')
df_merged.head(3)

,pair_id,comment,Komentar,code,pretvaranje int u string,red sa prioritetom,pretvaranje string u datum,sortiranje string liste,čuvanje liste u datoteku,postgresql konekcija,...,slanje binarnih podataka preko seriske veze,otpakovanje podataka iz tekstualne datoteke,pozicije podstingova u stringu,čitanje elemenata iz html-a - <td>,oduzimanje medijana iz svake kolone,uklanjanja zaglavlja prilikom spajanja nekoliko datoteka,parsiranje query stringa u url-u,rangiranje fazi članova na osnovu stepena podudaranja,izlaz u html datoteku,kako efikasno pročitati .csv datoteku
0,BookStackApp_BookStack_ActivityService_740,Get a new activity instance for the current u...,Daj novu instancu aktivnosti za trenutnog kori...,protected function newActivityForUser(stri...,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,BookStackApp_BookStack_CommentRepo_753,Update an existing comment.\n,Osveži postojeći komentar.,"public function update(Comment $comment, s...",0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,BookStackApp_BookStack_CommentRepo_754,Delete a comment from the system.\n,Obriši komentar iz sistema.,public function delete(Comment $comment) ...,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Prebacivanje podataka u oblik pogodan za klasifikaciju
Podaci u do sada korišćenoj tabeli su zgodni iz više razloga, ali nisu u formi traženoj u zadatku. Iz forme gde jedan red predstavlja jedan par Komentar-blok, podaci se transforimišu u format gde jedan red predstavlja jedan par komentar-upit

In [10]:
# Transformisanje podataka u oblik prigodan za klasifikaciju
import utility.preprocessing as preprocessing

df = preprocessing.transform_annotated_dataframe(df_merged)
df.head(3)

,PairID,QueryID,Comment,Query,Score
0,BookStackApp_BookStack_ActivityService_740,0,Daj novu instancu aktivnosti za trenutnog kori...,red sa prioritetom,0
1,BookStackApp_BookStack_ActivityService_740,1,Daj novu instancu aktivnosti za trenutnog kori...,pretvaranje string u datum,0
2,BookStackApp_BookStack_ActivityService_740,2,Daj novu instancu aktivnosti za trenutnog kori...,sortiranje string liste,0


In [11]:
df_lowercase = preprocessing.normalize_to_lowercase(df)

In [12]:
df_lowercase.head()

,PairID,QueryID,Comment,Query,Score
0,BookStackApp_BookStack_ActivityService_740,0,daj novu instancu aktivnosti za trenutnog kori...,red sa prioritetom,0
1,BookStackApp_BookStack_ActivityService_740,1,daj novu instancu aktivnosti za trenutnog kori...,pretvaranje string u datum,0
2,BookStackApp_BookStack_ActivityService_740,2,daj novu instancu aktivnosti za trenutnog kori...,sortiranje string liste,0
3,BookStackApp_BookStack_ActivityService_740,3,daj novu instancu aktivnosti za trenutnog kori...,čuvanje liste u datoteku,0
4,BookStackApp_BookStack_ActivityService_740,4,daj novu instancu aktivnosti za trenutnog kori...,postgresql konekcija,0
